#  Tradução de Textos - Experimento
## Utilização do modelo [MarianMT](https://huggingface.co/transformers/model_doc/marian.html) para tradução. 


* Neste exempo a tradução é feito do inglês para o português, mas ela pode ser feita em qualquer uma das línguas suportadas pelo MarianMT. 
* Para adaptar para traduções em outras línguas é necessário verificacar se há o modelo pré treinado disponível no MarianMT e adaptar o truncamento de strings do [spacy](https://spacy.io/usage/models) para o idioma desejado
* A métrica computada é o [sacrebleu](https://https://github.com/mjpost/sacrebleu) 


### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
O parâmetro `dataset` identifica os conjuntos de dados. Você pode importar arquivos de dataset com o botão  na barra de ferramentas.

In [49]:
dataset = "/tmp/data/paracrawl_en_pt_test.xlsx" #@param {type:"string"}
target = "target" #@param {type:"string", label:"Atributo alvo", description:"Seu modelo será treinado para prever os valores do alvo."}
prefix = ">>pt_br<<"  #@param [">>fr<<", ">>es<<", ">>it<<", ">>pt<<", ">>pt_br<<", ">>ro<<", ">>ca<<", ">>gl<<", ">>pt_BR<<",">>la<<", ">>wa<<", ">>fur<<", ">>oc<<", ">>fr_CA<<", ">>sc<<", ">>es_ES<<", ">>es_MX<<", ">>es_AR<<", ">>es_PR<<", ">>es_UY<<", ">>es_CL<<", ">>es_CO<<", ">>es_CR<<", ">>es_GT<<", ">>es_HN<<", ">>es_NI<<", ">>es_PA<<", ">>es_PE<<", ">>es_VE<<", ">>es_DO<<", ">>es_EC<<",">>es_SV<<", ">>an<<", ">>pt_PT<<",">>frp<<", ">>lad<<", ">>vec<<", ">>fr_FR<<", ">>co<<", ">>it_IT<<", ">>lld<<", ">>lij<<", ">>lmo<<", ">>nap<<", ">>rm<<", ">>scn<<", ">>mwl<<"] {type:"string",label:"Idioma de destino"}

# selected features to perform the model
filter_type = "incluir" #@param ["incluir","remover"]  {type:"string",label:"Modo de seleção das features", description:"Se deseja informar quais features deseja incluir no modelo, selecione a opção [incluir]. Caso deseje informar as features que não devem ser utilizadas, selecione [remover]. "}
model_features = "text" #@param {type:"string",multiple:true,label:"Features para incluir/remover no modelo",description:"Seu modelo será feito considerando apenas as features selecionadas. Caso nada seja especificado, todas as features serão utilizadas"}

#Hyperparams
model_name = "Helsinki-NLP/opus-mt-en-ROMANCE"  #@param ["Helsinki-NLP/opus-mt-NORTH_EU-NORTH_EU","Helsinki-NLP/opus-mt-ROMANCE-en","Helsinki-NLP/opus-mt-SCANDINAVIA-SCANDINAVIA","Helsinki-NLP/opus-mt-de-ZH","Helsinki-NLP/opus-mt-en-CELTIC","Helsinki-NLP/opus-mt-en-ROMANCE","Helsinki-NLP/opus-mt-es-NORWAY","Helsinki-NLP/opus-mt-fi-NORWAY","Helsinki-NLP/opus-mt-fi-ZH","Helsinki-NLP/opus-mt-fi_nb_no_nn_ru_sv_en-SAMI","Helsinki-NLP/opus-mt-sv-NORWAY","Helsinki-NLP/opus-mt-sv-ZH"] {type:"string",label:"Modelo pré treinado no idioma de origem"}
seed = 7 #@param {type:"integer",label"Semente de aleatoriedade"}
input_max_length = 127 #@param {type:"integer",label"Tamnho máximo da sentença de entrada"}
output_max_length = 256 #@param {type:"integer",label"Tamnho máximo da sentença de saída"}
inference_batch_size = 2 #@param {type:"integer",label"Tamanho do Batch de inferência"}

## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [63]:
import pandas as pd

df = pd.read_excel(dataset)

## Acesso aos metadados do conjunto de dados

Utiliza a função `stat_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar metadados.<br>
Por exemplo, arquivos CSV possuem `metadata['featuretypes']` para cada coluna no conjunto de dados (ex: categorical, numerical, or datetime).

In [64]:
from platiagro import stat_dataset

columns = df_test.columns.to_numpy()
target_index = np.argwhere(columns == target)
columns = np.delete(columns, target_index)

## Remoção de linhas com valores faltantes no atributo alvo
Caso haja linhas em que o atributo alvo contenha valores faltantes, é feita a remoção dos casos faltantes.


In [65]:
df.dropna(subset = [target],inplace=True)
y = df[target].to_numpy()

## Filtragem das features


In [53]:
if filter_type == 'incluir':
    if len(model_features) >= 1:
        columns_index = (np.where(np.isin(columns,model_features)))[0]
        columns_index.sort()
        columns_to_filter = columns[columns_index]
        #featuretypes = featuretypes[columns_index]
    else:
        columns_to_filter = columns
else:
    if len(model_features) >= 1:
        columns_index = (np.where(np.isin(columns,model_features)))[0]
        columns_index.sort()
        columns_to_filter = np.delete(columns,columns_index)
        #featuretypes = np.delete(featuretypes,columns_index)
    else:
        columns_to_filter = columns

# keep the features selected
df = df[columns_to_filter]
X= df.to_numpy()
X

array([['In this way, the civil life of a nation matures, making it possible for all citizens to enjoy the fruits of genuine tolerance and mutual respect.'],
       ['1999 XIII. Winnipeg, Canada July 23 to August 8'],
       ["In the mystery of Christmas, Christ's light shines on the earth, spreading, as it were, in concentric circles."],
       ['making it viable to drill two new boreholes in the west of that peninsula. ']],
      dtype=object)

## Verificando as configurações do MarianMT

Verificando disponibilidade de GPU e status de hardware

In [54]:
from multiprocessing import cpu_count
import torch
dev = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(dev)
if dev == "cpu":
    print(f"number of CPU cores: {cpu_count()}")
else:
    print(f"GPU: {torch.cuda.get_device_name(0)}, number of CPU cores: {cpu_count()}")

number of CPU cores: 12


Instanciando modelo e tokenizador

In [55]:
from transformers import MarianMTModel, MarianTokenizer

tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

Opções de tradução de idiomas

In [56]:
print(tokenizer.supported_language_codes)

['>>fr<<', '>>es<<', '>>it<<', '>>pt<<', '>>pt_br<<', '>>ro<<', '>>ca<<', '>>gl<<', '>>pt_BR<<', '>>la<<', '>>wa<<', '>>fur<<', '>>oc<<', '>>fr_CA<<', '>>sc<<', '>>es_ES<<', '>>es_MX<<', '>>es_AR<<', '>>es_PR<<', '>>es_UY<<', '>>es_CL<<', '>>es_CO<<', '>>es_CR<<', '>>es_GT<<', '>>es_HN<<', '>>es_NI<<', '>>es_PA<<', '>>es_PE<<', '>>es_VE<<', '>>es_DO<<', '>>es_EC<<', '>>es_SV<<', '>>an<<', '>>pt_PT<<', '>>frp<<', '>>lad<<', '>>vec<<', '>>fr_FR<<', '>>co<<', '>>it_IT<<', '>>lld<<', '>>lij<<', '>>lmo<<', '>>nap<<', '>>rm<<', '>>scn<<', '>>mwl<<']


Modelos pré treinados disponíveis

In [57]:
from transformers.hf_api import HfApi
model_list = HfApi().model_list()
org = "Helsinki-NLP"
model_ids = [x.modelId for x in model_list if x.modelId.startswith(org)]
suffix = [x.split('/')[1] for x in model_ids]
multi_models = [f'{org}/{s}' for s in suffix if s != s.lower()]
print(multi_models)

['Helsinki-NLP/opus-mt-NORTH_EU-NORTH_EU', 'Helsinki-NLP/opus-mt-ROMANCE-en', 'Helsinki-NLP/opus-mt-SCANDINAVIA-SCANDINAVIA', 'Helsinki-NLP/opus-mt-de-ZH', 'Helsinki-NLP/opus-mt-en-CELTIC', 'Helsinki-NLP/opus-mt-en-ROMANCE', 'Helsinki-NLP/opus-mt-es-NORWAY', 'Helsinki-NLP/opus-mt-fi-NORWAY', 'Helsinki-NLP/opus-mt-fi-ZH', 'Helsinki-NLP/opus-mt-fi_nb_no_nn_ru_sv_en-SAMI', 'Helsinki-NLP/opus-mt-sv-NORWAY', 'Helsinki-NLP/opus-mt-sv-ZH']


## Chamada da Classe MarianMT

In [58]:
hyperparams = {'input_max_length':input_max_length,'output_max_length':output_max_length,'inference_batch_size':inference_batch_size,'seed':seed}
model_params = {'model_name':model_name,'prefix':prefix}

In [59]:
!wget https://raw.githubusercontent.com/platiagro/tasks/main/tasks/nlp-marianmt-translator/marianmt_model.py

--2020-10-27 15:51:02--  https://raw.githubusercontent.com/platiagro/tasks/main/tasks/nlp-marianmt-translator/marianmt_model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.92.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.92.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6628 (6.5K) [text/plain]
Saving to: ‘marianmt_model.py.1’

marianmt_model.py.1 100%[===================>]   6.47K  --.-KB/s    in 0s      

2020-10-27 15:51:02 (37.7 MB/s) - ‘marianmt_model.py.1’ saved [6628/6628]



In [60]:
from marianmt_model import MarianMTTranslator
marian_model = MarianMTTranslator(hyperparams,model_params)
marian_model.get_result_dataframe(X,y)

100%|██████████| 2/2 [06:03<00:00, 181.80s/it]


,source_text,target_text,translated_text,bleu_score
0,"In this way, the civil life of a nation mature...","Deste modo, a vida civil de uma nação amadurec...","Desta forma, a vida civil de uma nação amadure...",64.856143
1,"1999 XIII. Winnipeg, Canada July 23 to August 8","1999 XIII. Winnipeg, Canadá 23 de julho a 8 de...","1999 XIII.Winnipeg, Canadá 23 de julho a 8 de ...",100.000000
2,"In the mystery of Christmas, Christ's light sh...","No mistério do Natal, a luz de Cristo irradia-...","No mistério do Natal, a luz de Cristo brilha n...",43.185757
3,making it viable to drill two new boreholes in...,e tem o objetivo de viabilizar a perfuração de...,Fazendo viável perfurar dois novos buracos no ...,6.208294


## Salva métricas

Utiliza a função `save_metrics` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar métricas. Por exemplo: `accuracy`, `precision`, `r2_score`, `custom_score` etc.<br>

In [61]:
from platiagro import save_metrics

save_metrics(avg_bleu=marian_model.avg_bleu)

## Salva modelo e outros resultados do treinamento

Escreve todos artefatos na pasta `/tmp/data/`. A plataforma guarda os artefatos desta pasta para usos futuros como implantação e comparação de resultados.

In [62]:
from joblib import dump

artifacts = {
    "hyperparams": hyperparams,
    "model_params": model_params,
    "columns":columns
}

dump(artifacts, "/tmp/data/ocr.joblib")

['/tmp/data/ocr.joblib']